<a href="https://colab.research.google.com/github/Nantakxrn007/Qwen3/blob/main/Qwen3_HF_GradioWebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p>
    <img src="https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEhq9mgxolMjpOJUfISV_c8pSNPRj9k6rS7-PWkhMQXA6UmOMF4rGT1AiXr_whifoJ8kl3NMujlHEs50CKcFDtrkN5gkKDerSz2FNJzzX2j7Egv8ZPd7pejyQF5Llee2dvpDV6UMbN10HDZBiF8KXra5IgcLPqpA2PHh1PniV9tEil3U8ttSVG1rNp8zXk-i/w320-h85/hg-logo.png" alt="PDF" width="200" height="50">
</p>

https://huggingface.co/

In [1]:
#%%capture
%pip install -U bitsandbytes
%pip install -U gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [3]:
import os, torch

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

from IPython.display import Markdown, display

import transformers
transformers.logging.set_verbosity_error()

device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.backends.mps.is_available():
    device = "mps"
print(f"Using device : {device}")

Using device : cuda


### HF Login

In [ ]:
# from huggingface_hub import login
# from google.colab import userdata

# hf_token    = userdata.get('HF_TOKEN')
# login(token = hf_token)

# hf_token    = 'hf_IjhzzEyBctnkroFJWqRxDDaaIyKVHsX'
# login(token = hf_token)

### Base Model

In [4]:
# From Gdrive
#base_model     = '/content/drive/MyDrive/BaseModel/Meta-Llama-3.1-8B-Instruct'

# From Huggingface
#base_model   = 'Qwen/Qwen3-8B'
base_model   = 'Qwen/Qwen3-14B'

# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    load_in_8bit=False,
    device_map="auto",
    attn_implementation="eager",
)

# Load tokenizer
tokenizer              = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

PAD_TOKEN = "<|pad|>"
tokenizer.add_special_tokens({"pad_token": PAD_TOKEN})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/36.5k [00:00<?, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not in

...

model-00008-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

KeyboardInterrupt: 

### Inference Model

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
%%time
messages = [
    {
        "role": "user", "content": "คุณเป็นใคร ทำอะไรได้บ้าง"
    }
]
prompt  = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=False)
inputs  = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")
outputs = model.generate(
    **inputs,
    max_length=8192,
    num_return_sequences=1,
    temperature=0.1,
    repetition_penalty=1.2,
    no_repeat_ngram_size=4
    )
response    = tokenizer.decode(outputs[0], skip_special_tokens=True)
answer      = response.split("assistant")[1]
display(Markdown(f"{answer}"))
print(len(answer))

In [ ]:
# output_ids = outputs[0][len(inputs.input_ids[0]):].tolist()

# # parsing thinking content
# try:
#     # rindex finding 151668 (</think>)
#     index = len(output_ids) - output_ids[::-1].index(151668)
# except ValueError:
#     index = 0

# thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
# content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

# print("thinking content:", thinking_content)
# print("content:", content)

### Chat Template
https://huggingface.co/docs/transformers/main/en/chat_templating

In [ ]:
%%time
messages = [
    { "role": "system","content": "You are a friendly chatbot who always responds in the style of a programmer" },

    { "role": "user" , "content": "คุณเป็นใคร ทำอะไรได้บ้าง"},
 ]
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True ,enable_thinking=True)

inputs  = tokenizer(tokenized_chat, return_tensors='pt', padding=True, truncation=True).to("cuda")
outputs = model.generate(
    **inputs,
    max_length=1024,
    num_return_sequences=1,
    temperature=0.1,
    repetition_penalty=1.2,
    no_repeat_ngram_size=4
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
answer   = response.split("assistant")[1]
display(Markdown(f"{answer}"))
print(len(answer))

In [ ]:
torch.cuda.empty_cache()

### Gradio WebUI   https://www.gradio.app/
<p>
    <img src="https://www.gradio.app/_app/immutable/assets/gradio.CHB5adID.svg" alt="PDF" width="200" height="50">
</p>

In [ ]:
import re
import time
import torch
import requests
import gradio as gr

from threading import Thread
from gradio import FileData

from transformers import TextIteratorStreamer , BitsAndBytesConfig

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

EMOJI    = "🤖"
COLOR    = "black"

def bot_streaming(message, history, system_prompt="You are a helpful assistan and answer in Thai", max_new_tokens=250):
    txt = message["text"]
    messages = []

    # Use the user-provided system prompt
    messages.append({"role": "system", "content": system_prompt})

    # Append conversation history
    for msg in history:
        if isinstance(msg[0], str) and isinstance(msg[1], str):
            messages.append({"role": "user", "content": msg[0]})
            messages.append({"role": "assistant", "content": msg[1]})

    # Add current user message
    messages.append({"role": "user", "content": txt})

    texts = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    inputs = tokenizer(texts, return_tensors="pt").to("cuda")
    streamer = TextIteratorStreamer(tokenizer, skip_special_tokens=True, skip_prompt=True)

    generation_kwargs = dict(
        inputs,
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        top_p=0.8,
        top_k=20,
        min_p=0.0,
        repetition_penalty=1.2,
        no_repeat_ngram_size=4
    )
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    buffer = ""
    for new_text in streamer:
        buffer += new_text
        time.sleep(0.01)
        yield buffer

# Gradio Interface
demo = gr.ChatInterface(
    fn=bot_streaming,
    title= EMOJI +" ChatBot by Qwen3 with Gradio UI ",
    textbox=gr.MultimodalTextbox(),
    additional_inputs=[
        gr.Textbox(
            value="You are a helpful assistant.",
            label="System Prompt"
        ),
        gr.Slider(
            minimum=1024,
            maximum=8192,
            value=2048,
            step=1,
            label="Maximum number of new tokens to generate"
        )
    ],
    cache_examples=False,
    description="Qwen3 with Gradio WebUI Chatbot Demo",
    stop_btn="Stop Generation",
    fill_height=True,
    multimodal=False
)

demo.launch(debug=True, share=True)
